In [1]:
import os
import sys
import json

from argparse import ArgumentParser

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.llms import ChatGLM
from langchain.chains import LLMChain, RetrievalQA
from src.utils import query_db

endpoint_url = ("http://127.0.0.1:29501")

llm = ChatGLM(
    endpoint_url=endpoint_url,
    history=[],
    model_kwargs={"sample_model_args": False},
)

/home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
from fuzzywuzzy import process, fuzz

def adjusted_ratio_fn(str1, str2):
    # 根据字符串长度计算一个权重，用于调整相似度得分
    len_weight = min(len(str1), len(str2)) / max(len(str1), len(str2))
    
    # 计算标准的相似度得分
    similarity_score = fuzz.WRatio(str1, str2)
    
    # 根据长度权重调整得分
    score = similarity_score * len_weight
    return score

process.extractOne(query="开启陡坡缓降系统后，组合仪表显示白色指示灯" , choices=["开启陡坡缓降系统后，组合仪表显示白色指示灯。\n"], scorer=adjusted_ratio_fn)


('开启陡坡缓降系统后，组合仪表显示白色指示灯。\n', 100.0)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/home/lzw/.hf_models/chatglm3-6b/"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to("cuda:3")

Loading checkpoint shards: 100%|██████████| 7/7 [00:10<00:00,  1.53s/it]


In [ ]:
with open("data/测试问题.json", 'r', encoding='utf-8') as f:
    datas = json.load(f)

questions = [data['question'] for data in datas]

In [17]:
model_name = "/home/lzw/.hf_models/stella-base-zh-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu", } ,
    encode_kwargs={"normalize_embeddings": False})

sentence_db = FAISS.load_local('vector_store/sentence', embeddings)

No sentence-transformers model found with name /home/lzw/.hf_models/stella-base-zh-v2. Creating a new one with MEAN pooling.


In [31]:
from fuzzywuzzy import fuzz, process
def adjusted_ratio_fn(str1, str2):
    # 根据字符串长度计算一个权重，用于调整相似度得分
    len_weight = min(len(str1), len(str2)) / max(len(str1), len(str2))
    
    # 计算标准的相似度得分
    similarity_score = fuzz.WRatio(str1, str2)
    
    # 根据长度权重调整得分
    score = similarity_score * len_weight
    return score

# best_match = process.extractOne(query, candidate_key, scorer=adjusted_ratio_fn)

/home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [41]:
from langchain.chains import create_citation_fuzzy_match_chain
retriever = sentence_db.as_retriever(search_type="similarity", search_kwargs={'k': 10})

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever,return_source_documents=True)

query = "如何从锁定状态唤醒中央显示器"

candidate_strs = [doc.page_content for doc in retriever.get_relevant_documents(query=query)]
matchs = process.extractBests(query, candidate_strs, scorer=adjusted_ratio_fn)
print(matchs)

matchs = [match[0] for match in matchs]
query = f"根据相关信息回答问题。问题是：“{query}” \n相关信息是：{'<SEP>'.join(matchs)}。答案是"
response, history = model.chat(tokenizer, query, history=[])
response
# result = qa({"query": query})

[('调节中央显示屏亮度在中央显示屏中点击-设置-显示，进入显示设置界面。\n01点击设置中央显示屏亮暗模式。', 7.280000000000001), ('调节中央显示屏亮度在中央显示屏中点击-设置-显示，进入显示设置界面。\n01点击设置中央显示屏亮暗模式。', 7.280000000000001), ('■仪表板上的调光器，请参见调节背光亮度（页码 90）。\n设置中央显示屏显示状态\n调节中央显示屏显示模式在中央显示屏中点击-设置-显示，进入显示设置界面。', 4.918918918918919), ('按压按钮：■短按暂停或播放音乐/视频。■长按约5秒锁屏/待机模式，在锁屏模式下，点击中央显示屏上的任意位置或长按中控台按钮可以重新解锁屏幕。\n主界面中央显示屏主界面按键01HOME按键：进入主界面。', 4.571428571428571), ('01点击设置中央显示屏亮暗模式。\n02滑动滑条调节中央显示屏亮度。您还可以通过以下方式调节中央显示屏亮度：■中央显示屏车辆功能界面，请参见车辆功能界面（页码 266）。', 4.439024390243903)]


'唤醒中央显示器的方法有多种：\n1. 调节中央显示屏亮度：在中央显示屏中点击-设置-显示，进入显示设置界面，然后点击设置中央显示屏亮暗模式。\n2. 调节中央显示屏显示状态：在中央显示屏中点击-设置-显示，进入显示设置界面，然后按压按钮，短按暂停或播放音乐/视频，长按约5秒锁屏/待机模式。\n3. 进入主界面：在主界面按键01HOME按键，进入主界面。\n\n这些方法都可以用来唤醒中央显示器。'

In [16]:
from tqdm import tqdm
system_info = {"role": "system", "content": "你是一个关键词提取器，下列是一个问题，需要你提取出文本中的关键词，不要回答问题！"}
history = [system_info]
related_str = [
    "激活陡坡缓降系统时，组合仪表显示绿色指示灯。\n当陡坡缓降系统出现故障时，组合仪表显示黄色指示灯。\n<SEP>警告:\n■在下坡路段，当陡坡缓降系统出现故障时，请立刻踩下制动踏板主动控制下坡速度。\n加油",
    "车身稳定控制系统关闭指示灯：车身稳定控制系统关闭时，该指示灯点亮。\n陡坡缓降系统指示灯：陡坡缓降系统开启并未激活时，白色指示灯点亮；激活后，绿色指示灯点亮。",
    "□激活HDC后，您可以通过踩下制动踏板或油门踏板调整下坡车速。\n功能状态说明开启陡坡缓降系统后，组合仪表显示白色指示灯。\n激活陡坡缓降系统时，组合仪表显示绿色指示灯。"
    "陡坡缓降系统指示灯：陡坡缓降系统开启并未激活时，白色指示灯点亮；激活后，绿色指示灯点亮。\n外部充电连接警告指示灯：外部充电电缆连接成功后，该指示灯点亮。",
]
tmp_str = [f"第{i+1}段相关信息" + related_str[i] for i in range(len(related_str))]

template = "你是一个关键词提取器，下列是一个问题，需要你提取出文本中的关键词。不要回答问题！最多提取3个关键词。\n问题是：“{question}” 将提取出的关键词以逗号隔开。\n关键词是："
query = f"根据问题，去除无关文本后，对相关文本按相关性进行排序，问题是：“开启陡坡缓降系统后，组合仪表显示什么颜色的指示灯？” \n相关信息是：{tmp_str}。对文本相似度排序："
response, history = model.chat(tokenizer, query, history=[])
response
# question_keyword_dict = {}
# for question in tqdm(questions):
#     query = template.format(question=question)
#     response, history = model.chat(tokenizer, query, history=[])
#     question_keyword_dict[question] = response

'1. 开启陡坡缓降系统后，组合仪表显示绿色指示灯。\n2. 当陡坡缓降系统出现故障时，组合仪表显示黄色指示灯。\n3. 加油\n4. 车身稳定控制系统关闭指示灯：车身稳定控制系统关闭时，该指示灯点亮。\n5. 陡坡缓降系统指示灯：陡坡缓降系统开启并未激活时，白色指示灯点亮；激活后，绿色指示灯点亮。\n6. □激活HDC后，您可以通过踩下制动踏板或油门踏板调整下坡车速。\n7. 功能状态说明开启陡坡缓降系统后，组合仪表显示白色指示灯。\n8. 激活陡坡缓降系统时，组合仪表显示绿色指示灯。\n9. 陡坡缓降系统指示灯：陡坡缓降系统开启并未激活时，白色指示灯点亮；激活后，绿色指示灯点亮。\n10. 外部充电连接警告指示灯：外部充电电缆连接成功后，该指示灯点亮。'

In [15]:
question_keyword_dict

{'怎样加热座椅？': '加热, 座椅, 方法',
 '自动模式下，中央显示屏是如何切换日间和夜间模式的？': '自动模式, 中央显示屏, 日间和夜间模式',
 '如何通过中央显示屏进行副驾驶员座椅设置？': '中央显示屏, 副驾驶员座椅设置',
 '副仪表台按钮如何操作中央显示屏？': '副仪表台按钮, 中央显示屏, 操作',
 '如何从锁定状态唤醒中央显示器?': '唤醒, 中央显示器, 状态',
 '如何正确使用颈椎保护系统？': '颈椎保护系统, 使用, 正确',
 '前方交叉路口预警系统（FCTA）的作用是什么？': '前方交叉路口预警系统（FCTA），作用',
 '在使用FCTA时需要注意哪些事项？': 'FCTA, 注意事项',
 '如何打开车辆尾门？': '打开, 车辆, 尾门',
 '在哪些情况下智能钥匙可能会受到干扰，导致功能异常？': '智能钥匙, 干扰, 功能异常',
 '车辆尾门的防夹保护功能是如何工作的？': '车辆尾门, 防夹保护功能, 工作原理',
 '在操作电动后备厢时需要注意哪些事项？': '电动后备厢, 操作, 注意事项',
 '如何进入车辆功能界面？': '车辆功能界面, 进入, 关键词',
 '在车辆功能界面有哪些操作选项？': '车辆功能, 操作选项',
 '如何编辑快捷开关图标?': '如何编辑, 快捷开关, 图标',
 '如何减少车辆腐蚀风险？': '减少车辆腐蚀风险,车辆腐蚀,风险',
 '如何通过空调系统面板调节空调风量？': '空调系统, 面板, 风量',
 '如何创建新的Lynk&CoID？': '创建新的Lynk&CoID, 关键词提取, 问题',
 '什么是车主账户？': '车主账户, 什么是, 车主',
 '如何创建人脸识别？': '创建, 人脸识别, 如何',
 '如何添加亲情账号？': '添加, 亲情账号, 如何',
 '如何开启或关闭用车偏好自动同步？': '开启, 关闭, 用车偏好自动同步',
 '如何熄火我的车辆？': '熄火, 车辆, 如何',
 '如何通过遥控钥匙启动车辆？': '遥控钥匙, 启动车辆, 如何',
 '如果遥控钥匙电池电量低，我应该如何启动车辆？': '遥控钥匙, 电池电量低, 启动车辆',
 '如何调节外后视镜？': '调节, 外后视镜, 方法',
 '外部反光境显示物体距

In [20]:
from typing import List, Optional, Tuple
def clean_related_str(related_str:List, keyword: Optional[tuple], threshold=-80):

    """
        Input:
            related_str: List[str]
        Return:
            List[str]
    """
    if len(related_str) == 1:
        return related_str

    ## remove previous section (if it contains the keyword)
    if len(keyword) > 0 and keyword[0][1] > threshold:
        potential_section = keyword[0][0] + "\n"
        for i in range(len(related_str)):
            if potential_section in related_str[i]:
                related_str[i] = potential_section + potential_section.join(related_str[i].split(potential_section)[1:])
            
    ## remove duplicate
    tmp_str = "<BOS>。\n".join(related_str)
    parts = tmp_str.split("。\n")
    seen = set()
    deduplicated_parts = []
    # 去重，同时保证去重后的顺序不变
    for part in parts:
        if part not in seen:
            seen.add(part)
            deduplicated_parts.append(part)
    tmp_str = "。\n".join(deduplicated_parts)
    related_str = tmp_str.split("<BOS>")
    related_str = [str.strip("。\n") for str in related_str]

    ## adding concat sentence
    # for i in range(len(related_str)):
    #     related_str[i] += f"第{i}段相关材料" + related_str[i]
    for str in related_str:
        print(str)
    return related_str


In [25]:
related_str = [
  "超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一次，以提醒前方车辆。\n使用危险警告灯\n危险警告灯按键车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危险警告灯。\n<SEP>说明:\n□发生碰撞情况下，危险警告灯也将自动点亮。\n调节背光亮度\n使用调光旋钮调节亮度白天情况下，未开启整车背光联动时，转动调光旋钮仅调节开关背光亮度，开启整车背光联动时，转动调光旋钮可同时调节中央显示屏和开关背光的亮度。\n夜晚情况下，未开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏和开关背光的亮度，开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏、中央显示屏、开关背光的亮度。在中央显示屏中点击-设置-显示，进入显示设置界面。\n01点击开启/关闭整车背光联动。\n□外界环境光照较强时，您可能会看不见开关背光。。\n",
            "夜晚情况下，未开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏和开关背光的亮度，开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏、中央显示屏、开关背光的亮度。在中央显示屏中点击-设置-显示，进入显示设置界面。\n<SEP>说明:\n□外界环境光照较强时，您可能会看不见开关背光。\n设置车内氛围灯\n氛围灯用于确保驾驶过程中车内光线不会太暗，同时也可营造愉快的驾驶环境。"
]
keywords = [
    [
        "车辆功能界面",
        -67.70877699284065
    ],
    [
        "通过中央显示屏选择驾驶模式",
        -71.42315380117144
    ],
]

clean_related_str(related_str, keywords)

超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一次，以提醒前方车辆。
使用危险警告灯
危险警告灯按键车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危险警告灯。
<SEP>说明:
□发生碰撞情况下，危险警告灯也将自动点亮。
调节背光亮度
使用调光旋钮调节亮度白天情况下，未开启整车背光联动时，转动调光旋钮仅调节开关背光亮度，开启整车背光联动时，转动调光旋钮可同时调节中央显示屏和开关背光的亮度。
夜晚情况下，未开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏和开关背光的亮度，开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏、中央显示屏、开关背光的亮度。在中央显示屏中点击-设置-显示，进入显示设置界面。
01点击开启/关闭整车背光联动。
□外界环境光照较强时，您可能会看不见开关背光
<SEP>说明:
□外界环境光照较强时，您可能会看不见开关背光。
设置车内氛围灯
氛围灯用于确保驾驶过程中车内光线不会太暗，同时也可营造愉快的驾驶环境


['超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一次，以提醒前方车辆。\n使用危险警告灯\n危险警告灯按键车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危险警告灯。\n<SEP>说明:\n□发生碰撞情况下，危险警告灯也将自动点亮。\n调节背光亮度\n使用调光旋钮调节亮度白天情况下，未开启整车背光联动时，转动调光旋钮仅调节开关背光亮度，开启整车背光联动时，转动调光旋钮可同时调节中央显示屏和开关背光的亮度。\n夜晚情况下，未开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏和开关背光的亮度，开启整车背光联动时，转动调光旋钮可同时调节仪表显示屏、中央显示屏、开关背光的亮度。在中央显示屏中点击-设置-显示，进入显示设置界面。\n01点击开启/关闭整车背光联动。\n□外界环境光照较强时，您可能会看不见开关背光',
 '<SEP>说明:\n□外界环境光照较强时，您可能会看不见开关背光。\n设置车内氛围灯\n氛围灯用于确保驾驶过程中车内光线不会太暗，同时也可营造愉快的驾驶环境']